In [ ]:
#hide 
from IPython.core.debugger import set_trace

### Decorators without arguments

Higher order functions, or functions that return functions, can be used as decorators in Python.Decorators behave differently depending on if they have arguments or not. First let's look at how decorators without arguments work. 

In other words: 

```
@some_decorator
def add(x,y): return x + y 
```
is the same thing as 
```
def add(x,y): return x + y
add = some_decorator(add)
```

You call the function `some_decorator` on `add` and you assign the answer back to `add`.  

The structure of decorators looks like this 

In [ ]:
def some_decorator(f): 
    def inner(*args, **kwargs): 
        return f(*args, **kwargs)
    return inner

where `some_decorator` is the decorator, `f` is the function that is decorated (i.e. `add` from above), `*args` and `**kwargs` are arguments to `f`, and `inner` is the return value from `some_decorator(add)` that gets assigned back to `add`. 

The outside layer executes once when you decorate the function. The inner layer executes every time you call the function. So if you decorate `add` with `some_decorator`, every time you call `add` the `inner` function will run. 

![](./assets/double_layer_function.jpeg)

Below we have a function `changedoc`, and examples of how it can be used both as a decorator and by itself. 

In [ ]:
# changedoc is a function that changes documentation string of another function. 
def changedoc(f1): 
    f1.__doc__ = "a new doc"
    return f1

# Testing changedoc on some functions. 
# This one we use without a decorator. 
def somefun(x): return x+1
somefun = changedoc(somefun)
somefun.__doc__  # returns "a new doc"

@changedoc
def anotherfun(x): return x+2
anotherfun.__doc__  # returns "a new doc"

'a new doc'

#### Add a predefined string to the end of a function 

Here is another example. This decorator adds the word "hello" to the end of a function. The `*args` and `**kwargs` bit inside `inner` is useful because it holds any arguments you pass to `f`. 

A quick note. Printing `locals()` in a function call is useful for trying to debug decorators. You can see exactly what variables are present. 

In [ ]:
def add_hello(f): 
    def inner(*args, **kwargs): 
        print("Variables present: " , locals())
        return f(*args, **kwargs) + "_hello" 
    return inner

# Without decorator 
def print_alphabet(): return "abcdefghijklmnopqrstuvwxyz"
f_temp = add_hello(print_alphabet)
print(f_temp())

# With decorator
@add_hello
def print_alphabet(): return "abcdefghijklmnopqrstuvwxyz"
print(print_alphabet())

# Without decorator, and using args, kwargs
def repeat_x(x, n=4):    return x*n
f_temp = add_hello(repeat_x)
print(f_temp("deception ", n=6))

# With decorator, and using args, kwargs 
@add_hello
def repeat_x(x, n=4):    return x*n
print(repeat_x("deception ", n=6))

Variables present:  {'args': (), 'kwargs': {}, 'f': <function print_alphabet at 0x10c1c81e0>}
abcdefghijklmnopqrstuvwxyz_hello
Variables present:  {'args': (), 'kwargs': {}, 'f': <function print_alphabet at 0x10c285488>}
abcdefghijklmnopqrstuvwxyz_hello
Variables present:  {'args': ('deception ',), 'kwargs': {'n': 6}, 'f': <function repeat_x at 0x10c285f28>}
deception deception deception deception deception deception _hello
Variables present:  {'args': ('deception ',), 'kwargs': {'n': 6}, 'f': <function repeat_x at 0x10c2a3378>}
deception deception deception deception deception deception _hello


You can see how the decorator syntax is a bit cleaner than explicitly calling the function. But sometimes it isn't intuitive what a decorator is doing, so be careful. 

#### Time a function 

Here is a decorator used for timing functions. The function `add_stuff` goes into the `f` argument for `time_it`. The arguments `x` and `y` in `add_stuff` are caught by `*args` in `wrapper` and are accessed in a tuple in the `args` variable. 

In [ ]:
import time 
def time_it(f): 
    def inner(*args): 
        t = time.time()
        result = f(*args)
        print (f.__name__ + " takes " +  str(time.time() - t) + " seconds.")
        return result 
    return inner

@time_it
def add_stuff(x,y): return x+y
add_stuff(1,2)

add_stuff takes 1.9073486328125e-06 seconds.


3

#### Log the internal state of a function 

Here is one for logging. Pretend the `print` statements actually writes to a file, and you get the idea. 

In [ ]:
def add_logs(f): 
    def inner(*args, **kwargs): 
        # insert real logging code here 
        print('write', *args, "to file")
        return f(*args, **kwargs)
    return inner    

@add_logs
def add_things(x,y,z): return x+y+z
add_things(1,4,3)

write 1 4 3 to file


8

#### Set a time limit on how often a function can be called 

Say we wanted a function to be called no more than once every minute. We can do this with decorators too. 

In [ ]:
def once_per_min(f): 
    calltime = 0 
    def inner(*args, **kwargs): 
        nonlocal calltime
        gap = time.time() - calltime
        if gap < 60: 
            msg = "You're calling this function too often. Try again in " + \
                          str(60 - gap) + " seconds."
            raise Exception(msg)
        calltime = time.time()
        return f(*args, **kwargs)
    return inner

In [ ]:
@once_per_min
def add_stuff(x,y,z): return x+y+z

In [ ]:
add_stuff(1,2,3)

6

In [ ]:
#add_stuff(1,2,3)

#### Make a function print out useful debugging information 

In [ ]:
import inspect
def debug_this(f): 
    def inner(*args, **kwargs): 
        print("Arguments of f:", *args)
        print("Keyword arguments of f:", **kwargs)
        print("Result of f:", f(*args, **kwargs))
        print("f.__dict__:", f.__dict__)
        print("Source of f: \n####\n", inspect.getsource(f), "####")
        return f(*args, **kwargs)
    return inner

In [ ]:
@debug_this
def somefun(a,b,c): 
    """This function is a bit complex but doesn't do anything interesting"""
    d = a + b 
    e = b + c 
    f = 10
    for x in (a,b,c,d,e): 
        f += x 
    return f 

In [ ]:
somefun(1,2,4)

Arguments of f: 1 2 4
Keyword arguments of f:
Result of f: 26
f.__dict__: {}
Source of f: 
####
 @debug_this
def somefun(a,b,c): 
    """This function is a bit complex but doesn't do anything interesting"""
    d = a + b 
    e = b + c 
    f = 10
    for x in (a,b,c,d,e): 
        f += x 
    return f 
 ####


26

In [ ]:
inspect.getmodule(somefun)

<module '__main__'>

### Decorators with arguments 

Decorators can be used with arguments. But the structure of these decorators is different. They need three layers, not two.

In [ ]:
def some_decorator(n): 
    def middle(f): 
        def inner(*args, **kwargs): 
            return f(*args, **kwargs)
        return inner  
    return middle 

The outer layer is the name of the decorator, here `once_per_n`. The parameter to the decorator is also the parameter of the outer layer, which here I've called `n`. The outer layer executes once, when you provide an argument to the decorator. 
The middle layer has as parameter `f`, the decorated function. This middle layer executes once, when we decorate the function. 
The inner layer has as arguments `*args` and `**kwargs`, the arguments to the decorated function `f`. This inner layer is executed any time that the decorated function `f` is called. 

Thanks to Reuven Lerner for this diagram: 

![](./assets/triple_layer_function.jpeg)

#### Stop a function running more than every n seconds 

Let's say we had a decorator `@once_per_n`. Say it works like the `@once_per_min` decorator above, except it stopped functions running every `n` seconds (instead of every minute) where `n` is some argument we give to the decorator. 
Then

```
@once_per_n(5)
def add(x,y): return x+y
```
is the same as 
```
add = once_per_n(5)(add)
```

The `once_per_n(5)`function also returns a function, that is then called on `add`. So what does the structure of `@once_per_n` look like? 

In [ ]:
def once_per_n(n): 
    def middle(f):
        calltime = 0 
        def inner(*args, **kwargs): 
            nonlocal calltime; 
            gap = time.time() - calltime 
            if gap < n: 
                msg = "You're calling this function too often. Try again in " + \
                          str(n - gap) + " seconds."
                raise Exception(msg)
            calltime = time.time()
            return f(*args, **kwargs)
        return inner  
    return middle 

In [ ]:
@once_per_n(5)
def add(x,y): return x+y

In [ ]:
add(5,1)

6

In [ ]:
#add(65,15)

Let's see some applications of this. 

#### Add any string to the output of a function 

Unlike before, we can pass any string we like as a parameter to the decorator, and add that string to the end of a function  

In [ ]:
# A decorator to add a word to the output of a function 
def add_word(word): 
    def middle(f): 
        def inner(*args, **kwargs):
            return f(*args, **kwargs) + word
        return inner
    return middle 

In [ ]:
@add_word("_hello")
def alphabet(): return 'abcdefghijklmnopqrstuvwxyz'
print(alphabet())

@add_word("oh boy!!!")
def repeat_x(x, n=2): return x*n 
print(repeat_x("deception ", n=4))

abcdefghijklmnopqrstuvwxyz_hello
deception deception deception deception oh boy!!!


#### Make a numerical function return modulo n

In [ ]:
def mod_n(n): 
    def middle(f): 
        def inner(*args, **kwargs):
            return f(*args, **kwargs) % n
        return inner 
    return middle 


@mod_n(7)
def add(x,y): return x + y
print(add(5,1), add(5,2), add(5,3))

6 0 1


#### Run tests when a function is defined

This example runs some unit tests against a function the first time you define it. This is useful to see if you make a mistake or not. Also if you change something in definition of the function, the tests will run automatically. 

In [ ]:
def run_tests(tests): 
    def middle(f): 
        for params, result in tests: 
            if f(*params) == result: print("Test", *params, "passed.")
            else:                    print("Test", *params, "failed.")
        def inner(*args, **kwargs):
            return f(*args, **kwargs)
        return inner 
    return middle 

tests_eq_10 = [
    [(1,2,4), False],
    [(1,2,7), True],
    [(10,0,0),True], 
    [(-10,10,10),True],
    [(4,0,7), False]
]
@run_tests(tests_eq_10)
def adds_to_ten(x,y,z): return True if x+y+z==10 else False 

tests_odd = [
    [(1,3,4), False], 
    [(1,3,5), True], 
    [(0,0,0), False], 
    [(0,0,1), True]
]
@run_tests(tests_odd)
def adds_to_odd(x,y,z): return (x+y+z)%2==1

Test 1 2 4 passed.
Test 1 2 7 passed.
Test 10 0 0 passed.
Test -10 10 10 passed.
Test 4 0 7 passed.
Test 1 3 4 passed.
Test 1 3 5 passed.
Test 0 0 0 passed.
Test 0 0 1 passed.


#### Inherit documentation from another function

Making a function inherit documentation from another function is a useful task that can be done with decorators. In this example our decorator is `copy_docs`, the function with documentation we want to copy `f_with_docs`, and the decorated function `f`. 

Note that there is no `inner` function below, because we don't need to ever execute `f`. We are just copying across documentation and leaving it otherwise unchanged. 

In [ ]:
def copy_docs(f_with_docs):
    def middle(f):
        f.__doc__  = f_with_docs.__doc__
        f.__name__ = f_with_docs.__name__
        return f
    return middle

import numpy as np
@copy_docs(np.add)
def add(x,y): return x+y
print(add.__doc__[0:300])  # same documentation as numpy add function 

add(x1, x2, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

Add arguments element-wise.

Parameters
----------
x1, x2 : array_like
    The arrays to be added.  If ``x1.shape != x2.shape``, they must be
    broadcastable to a common shape (whi


### References / Further Reading

* Reuven Lerner's talk: [Slides](https://speakerdeck.com/pycon2019/reuven-m-lerner-practical-decorators) and [Talk](https://www.youtube.com/watch?v=MjHpMCIvwsY&feature=youtu.be)
* [Common uses of Python Decorators](https://stackoverflow.com/questions/489720/what-are-some-common-uses-for-python-decorators)
* [Decorators with Parameters](https://stackoverflow.com/questions/5929107/decorators-with-parameters)